In [ ]:
import pandas as pd
import numpy as np
from time import time
import sys, io
import larch as lx
from pytest import approx

In [ ]:
import larch as lx

In [ ]:
varnames = [
    'price', 'time', 'conven', 'comfort', 'meals', 'petfr',
    'emipp', 'nonsig1', 'nonsig2', 'nonsig3',
]
d = lx.examples.ARTIFICIAL()
m = lx.Model(d)
m.utility_ca = sum(lx.PX(i) for i in varnames)
m.choice_ca_var = 'choice'
randvars_normal = ['meals', 'petfr', 'emipp']

In [ ]:
m.mixtures = [lx.mixtures.Normal(k, f"sd.{k}") for k in randvars_normal]

In [ ]:
m.n_draws = 300
m.seed = 42
m.prerolled = True
m.common_draws = True

In [ ]:
# TEST
assert m.loglike() == approx(-4394.45556640625)

In [ ]:
# TEST
assert m.d_loglike() == approx([ 
    2.085461e+03, -1.130608e+03, -5.535062e+03,  1.998748e+02, -3.927801e+03,  3.330141e+03, -4.460615e+03,
   -1.380896e+03, -1.267620e+03, -3.135025e+00, -3.900630e+00, -1.376858e+00, -2.561676e+03], rel=1e-4)

In [ ]:
# TEST
se, hess, ihess = m.jax_param_cov(m.pvals)
assert hess == approx(np.array([ 
             [ 5.214861e+03, -8.737501e+03, -1.207186e+04,  4.689243e+03, -4.701574e+03,  2.698417e+03, -2.037047e+03,
              -6.022355e+03, -8.715623e+03,  4.149471e+00, -1.291960e+01, -1.636162e-01,  2.693130e+03],
             [-8.737502e+03,  1.918251e+04,  2.199170e+04, -1.065871e+04,  6.688360e+03, -2.722472e+03,  4.219213e+01,
               1.198888e+04,  1.867886e+04,  5.044937e+00,  2.636754e+01,  8.628845e-01, -8.679251e+03],
             [-1.207186e+04,  2.199169e+04,  3.128377e+04, -1.174997e+04,  1.223288e+04, -7.146222e+03,  5.618984e+03,
               1.528325e+04,  2.193285e+04, -1.277624e+01,  3.275142e+01,  6.413689e-01, -6.471689e+03],
             [ 4.689244e+03, -1.065871e+04, -1.174997e+04,  6.683431e+03, -3.098420e+03,  9.166903e+02,  8.549390e+02,
              -6.647478e+03, -1.064853e+04, -4.171021e+00, -1.501747e+01, -1.888428e-01,  5.545634e+03],
             [-4.701574e+03,  6.688361e+03,  1.223288e+04, -3.098420e+03,  6.745899e+03, -4.474388e+03,  4.947241e+03,
               5.334855e+03,  6.653889e+03, -1.206940e+01,  1.123395e+01,  3.519630e-01,  2.137472e+02],
             [ 2.698417e+03, -2.722472e+03, -7.146222e+03,  9.166902e+02, -4.474388e+03,  4.030793e+03, -4.448886e+03,
              -2.682003e+03, -2.698321e+03,  1.166885e+01, -5.420895e+00, -1.787338e-01, -1.729178e+03],
             [-2.037046e+03,  4.219213e+01,  5.618985e+03,  8.549389e+02,  4.947241e+03, -4.448886e+03,  6.745437e+03,
               1.371551e+03,  3.171216e+01, -1.749387e+01,  2.293404e+00,  8.043671e-02,  4.169263e+03],
             [-6.022355e+03,  1.198888e+04,  1.528325e+04, -6.647478e+03,  5.334855e+03, -2.682003e+03,  1.371551e+03,
               8.437011e+03,  1.195731e+04,  1.625114e+00,  1.781580e+01,  7.714996e-01, -4.662336e+03],
             [-8.715623e+03,  1.867886e+04,  2.193285e+04, -1.064853e+04,  6.653889e+03, -2.698321e+03,  3.171217e+01,
               1.195731e+04,  1.910547e+04,  4.064438e+00,  2.614574e+01,  5.830841e-01, -8.672464e+03],
             [ 4.149475e+00,  5.044861e+00, -1.277600e+01, -4.171112e+00, -1.206927e+01,  1.166878e+01, -1.749390e+01,
               1.624878e+00,  4.064697e+00, -2.158142e+03,  1.105316e+02, -2.832360e+01, -1.466002e+01],
             [-1.291957e+01,  2.636751e+01,  3.275133e+01, -1.501750e+01,  1.123393e+01, -5.420914e+00,  2.293438e+00,
               1.781590e+01,  2.614552e+01,  1.105317e+02,  3.290158e+03,  1.039862e+02, -1.082738e+01],
             [-1.636047e-01,  8.629456e-01,  6.414795e-01, -1.887054e-01,  3.519135e-01, -1.787491e-01,  8.040237e-02,
               7.715759e-01,  5.830688e-01, -2.832359e+01,  1.039862e+02,  1.726320e+03, -4.051666e-01],
             [ 2.693130e+03, -8.679251e+03, -6.471688e+03,  5.545634e+03,  2.137472e+02, -1.729178e+03,  4.169263e+03,
              -4.662337e+03, -8.672464e+03, -1.466016e+01, -1.082734e+01, -4.051819e-01,  7.339009e+03]],            dtype=np.float32), rel=1e-3)

In [ ]:
# TEST
assert m._draws[0:2] == approx(np.array([[ 0.328483,  0.256732,  0.388444],
       [ 0.170979,  0.871594,  0.298636]], dtype=np.float32), rel=1e-5)
assert m._draws[-2:] == approx(np.array([[ 0.300983,  0.328402,  0.055866],
       [ 0.363251,  0.119268,  0.45048 ]], dtype=np.float32), rel=1e-5)

In [ ]:
result = m.maximize_loglike(stderr=True)

In [ ]:
m2 = m.copy()
m2.pvals = 'init'
m2.common_draws = False

In [ ]:
result2 = m2.maximize_loglike(stderr=True)

In [ ]:
# TEST
# changing seeds
m.seed = 45
assert m.loglike() == approx(-2279.95263671875)
m.seed = 46
assert m.loglike() == approx(-2282.3642578125)

In [ ]:
m2.parameter_summary()

In [ ]:
from xlogit import MixedLogit

df = d.to_dataframe().reset_index()
varnames = ['price', 'time', 'conven', 'comfort', 'meals', 'petfr',
            'emipp', 'nonsig1', 'nonsig2', 'nonsig3']
X = df[varnames].values
y = df['choice'].values
randvars = {'meals': 'n', 'petfr': 'n', 'emipp': 'n'}
alts = df['alt']
ids = df['id']
panels = None
batch_size = 5000
n_draws = 300

In [ ]:
np.random.seed(0)
model = MixedLogit()
model.fit(X, y, varnames, alts=alts, ids=ids, n_draws=n_draws,
          panels=panels,verbose=0, randvars=randvars,
          batch_size=batch_size)

In [ ]:
model.summary()